# 🎯 Week 3: Multi-Agent Orchestration

## Supervisor Pattern, Subgraphs & Shared State

Welcome to Week 3! Last week you built a single ReAct agent. Today, you'll build a **team of agents** that work together.

> **📚 Official Documentation:** This notebook follows the [LangGraph Official Documentation](https://docs.langchain.com/oss/python/langgraph/overview). All APIs and patterns used here are based on the current LangGraph specification.

---

## 📋 Recap & Today's Plan

**Last week:** Typed State, Conditional Edges, Checkpointing, Meeting Prep Agent

**Today:** Your Week 1 router + your Week 2 ReAct agent combine into a real multi-agent system.

*"Last week you built the employee. Today you build the team."*

**Today's agenda:** 
1. Supervisor Pattern → 
2. Shared vs Scoped State → 
3. Subgraphs → 
4. Live Build

---

## 🚀 What You'll Learn Today

1. **Why Multi-Agent?** — When and why to split work across agents
2. **Supervisor Pattern** — The coordinator that never does the work
3. **Shared vs Scoped State** — What each agent sees vs what stays private
4. **Subgraphs** — Agents as nodes in a parent graph
5. **Live Build** — Company Research Assistant with 3 agents

### 🎯 By the End of This Notebook

- ✅ Understand when to use multi-agent systems
- ✅ Implement the Supervisor Pattern in LangGraph
- ✅ Design shared and scoped state schemas
- ✅ Build subgraphs and compose them into parent graphs
- ✅ Create a working Company Research Assistant
- ✅ Debug common multi-agent issues

---

### 📖 How to Use This Notebook

1. **Run cells in order** - Each cell builds on the previous one
2. **Read the markdown cells** - They contain important explanations
3. **Experiment** - Try modifying the code to see what happens
4. **Complete the homework** - Extend the system with a third agent

**Ready? Let's start!**


## 🔧 Setup & Installation


## 🔑 API Key Setup (IMPORTANT!)

**To get REAL data instead of mock data, you need to set up API keys:**

1. **Create a `.env` file** in the `week-3` directory
2. **Add your API keys:**
   ```
   OPENAI_API_KEY=your_openai_api_key_here
   TAVILY_API_KEY=your_tavily_api_key_here
   ```

3. **Get your API keys:**
   - **OpenAI**: https://platform.openai.com/api-keys
   - **Tavily**: https://tavily.com (free tier available)

**Without API keys, the system will use mock data for demonstration purposes.**

You can copy `.env.example` to `.env` and fill in your keys:
```bash
cp .env.example .env
# Then edit .env with your actual keys
```


In [ ]:
# Install dependencies (run once)
%pip install langgraph langchain-core langchain-openai langchain ipython python-dotenv tavily-python

# Verify installation
import sys
print(f"Python version: {sys.version}")
print("✅ Dependencies installed successfully!")
print("💡 Make sure you have a .env file with OPENAI_API_KEY and TAVILY_API_KEY!")


## 📦 Imports


In [ ]:
# Core imports
import operator
from typing import Annotated, Literal
from typing_extensions import TypedDict
import os
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables
# Try loading from current directory first, then parent directory
env_path = Path(".env")
if not env_path.exists():
    env_path = Path("../.env")
if not env_path.exists():
    env_path = Path("../../.env")

load_dotenv(env_path)

# LangGraph imports
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

# LangChain imports
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

# Tavily Search import
try:
    from tavily import TavilyClient
    TAVILY_AVAILABLE = True
except ImportError:
    TAVILY_AVAILABLE = False
    print("⚠️  Tavily not installed. Install with: pip install tavily-python")

# Verify API keys
api_key = os.getenv("OPENAI_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")

print("=" * 60)
print("🔑 API Key Status:")
print("=" * 60)

if api_key:
    print("✅ OpenAI API key loaded")
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
else:
    print("❌ OPENAI_API_KEY not found")
    print("   📝 To fix: Create a .env file with:")
    print("      OPENAI_API_KEY=your_key_here")
    print("   🔗 Get your key: https://platform.openai.com/api-keys")
    llm = None

if tavily_key and TAVILY_AVAILABLE:
    print("✅ Tavily API key loaded")
    tavily_client = TavilyClient(api_key=tavily_key)
else:
    print("❌ TAVILY_API_KEY not found or Tavily not installed")
    if not TAVILY_AVAILABLE:
        print("   📝 Install Tavily: pip install tavily-python")
    print("   📝 To fix: Add to .env file:")
    print("      TAVILY_API_KEY=your_key_here")
    print("   🔗 Get your key: https://tavily.com (free tier available)")
    tavily_client = None

print("=" * 60)

if not api_key or not tavily_key:
    print("\n⚠️  WARNING: Missing API keys will result in MOCK DATA")
    print("   The system will work but return placeholder responses.")
    print("   To get real data, set up your API keys in a .env file.\n")
else:
    print("\n✅ All API keys configured! Ready for real data.\n")

print("✅ All imports successful!")


---

# 📚 Concept 1: Why Multi-Agent?

## Single Agents Break When:

- **Tasks have too many steps** — context overload
- **You need different expertise** for different parts
- **One failure takes everything down** — no isolation
- **You need to run things simultaneously** — parallel execution

## Multi-Agent Systems Give You:

| Benefit | Description |
|---------|-------------|
| **Specialization** | Each agent does one thing well |
| **Isolation** | Failures are contained |
| **Scalability** | Agents can run in parallel |
| **Maintainability** | Swap one agent without breaking the rest |

*"One agent, ten tasks = one person doing every job in the company."*

---

# 📚 Concept 2: The Supervisor Pattern

## The Supervisor Never Does the Work. It Only Decides Who Does.

```
User → Supervisor → Researcher → Supervisor → Writer → Supervisor (FINISH) → User
```

### How It Works:

1. Supervisor reads the task and current state
2. Outputs a routing decision: `researcher` / `writer` / `FINISH`
3. Specialist agent runs and writes results to shared state
4. Supervisor reads again and decides what's next
5. Repeat until FINISH

**The routing function** — same conditional edges from Week 2, now routing between agents instead of tools.

### Bad Supervisor Prompt:
```
You manage a team. Decide who goes next. Members: researcher, writer
```

### Good Supervisor Prompt:
```
Route to researcher if research is not done.
Route to writer if research is done but no draft exists.
Respond FINISH if both research and draft are complete.
Reply with exactly one word: researcher, writer, or FINISH.
```


---

# 📚 Concept 3: Shared vs Scoped State

## Shared State — Visible to Everyone

The supervisor reads this to make decisions. All agents can read/write shared state.

## Scoped State — Private to a Subgraph

Internal reasoning, intermediate results, tool call noise. Never reaches the supervisor.

### Example:

```python
# Shared — supervisor sees this
class SupervisorState(TypedDict):
    task: str
    research_summary: str   # clean output from researcher
    draft: str              # clean output from writer
    next: str               # routing decision

# Scoped — researcher only
class ResearchState(TypedDict):
    query: str
    raw_search_results: List[str]   # internal noise
    research_summary: str            # only this goes back up
```

**Rule of thumb:** Shared state = what the supervisor needs + what the user sees. Everything else = scoped.

**What breaks without it:** Every agent's internal tool calls, search results, and scratchpad pile into shared messages. Supervisor starts making routing decisions based on noise. Quality collapses.


---

# 📚 Concept 4: Subgraphs

## A Subgraph is a Compiled LangGraph App Used as a Single Node

From the parent's perspective: **black box**. Input in, output out.
Internally: its own state, nodes, edges, loops, tools.

### How It Works:

```python
# Build the subgraph
research_graph = StateGraph(ResearchState)
research_graph.add_node("search", search_node)
research_graph.add_node("summarize", summarize_node)
research_graph.add_edge(START, "search")
research_graph.add_edge("search", "summarize")
research_graph.add_edge("summarize", END)
research_agent = research_graph.compile()

# Drop it into the parent as a single node
parent_graph.add_node("researcher", research_agent)
```

**State handoff:** Fields that exist in both parent state and subgraph state are passed automatically. Only matching fields flow in and out.

**Your Week 2 Meeting Prep Agent is already a subgraph. It just doesn't know it yet.**


---

# 🏗️ What We're Building

## Company Research Assistant

**Three agents. One goal:** Type in a company name, get back a structured brief.

| Agent | Role |
|-------|------|
| **Supervisor** | Reads the task, decides who acts, synthesizes the final output |
| **Research Agent** | Runs web searches, returns a clean 3-paragraph summary |
| **Writer Agent** | Takes the summary, writes a structured brief: Overview / Recent Developments / Outreach Signals |

### Project Structure:

```
week-3/
├── state.py          # SupervisorState and ResearchState
├── agents/
│   ├── supervisor.py # Routing logic and FINISH condition
│   ├── researcher.py # Research subgraph with search + summarize
│   └── writer.py     # Writer node that drafts the brief
├── graph.py          # Wire all nodes, add conditional edges
└── main.py           # Invoke and print output
```

**Let's build it step by step!**


---

# 🔨 Live Build: Step 1 - Define State Schemas

## Shared State (SupervisorState)

This is what the supervisor sees and uses to make routing decisions.


In [ ]:
# Shared state - visible to supervisor and all agents
class SupervisorState(TypedDict):
    """State shared across all agents. Supervisor reads this to make decisions."""
    task: str                    # Company name to research
    research_summary: str        # Clean output from researcher (3 paragraphs)
    draft: str                  # Clean output from writer (structured brief)
    next: str                   # Routing decision: "researcher", "writer", or "FINISH"
    messages: Annotated[list, operator.add]  # Conversation history

print("✅ SupervisorState defined (shared state)")


## Scoped State (ResearchState)

This is private to the research subgraph. Only `research_summary` flows back to shared state.


In [ ]:
# Scoped state - private to research subgraph
class ResearchState(TypedDict):
    """State internal to the research subgraph. Only research_summary flows back up."""
    task: str                    # Matches SupervisorState.task (auto-passed)
    raw_search_results: list      # Internal noise - never seen by supervisor
    research_summary: str         # Clean output - flows back to SupervisorState
    messages: Annotated[list, operator.add]  # Internal conversation

print("✅ ResearchState defined (scoped state)")
print("💡 Note: 'task' and 'research_summary' match SupervisorState - they auto-flow!")


---

# 🔨 Live Build: Step 2 - Research Agent (Subgraph)

The research agent is a **subgraph** with two nodes:
1. **search_node** - Runs web searches using Tavily
2. **summarize_node** - Condenses results into 3 clean paragraphs


In [ ]:
# Tool: Web search using Tavily
@tool
def tavily_search(query: str) -> str:
    """Search the web for information about a company."""
    if not tavily_client:
        return f"[Mock] Search results for: {query}"
    
    try:
        response = tavily_client.search(
            query=query,
            search_depth="advanced",
            max_results=5
        )
        results = []
        for result in response.get("results", []):
            results.append(f"Title: {result.get('title', 'N/A')}\nContent: {result.get('content', 'N/A')}")
        return "\n\n---\n\n".join(results)
    except Exception as e:
        return f"Error searching: {str(e)}"

print("✅ Tavily search tool defined")


In [ ]:
# Node 1: Search node - runs web searches
def search_node(state: ResearchState) -> dict:
    """Search the web for information about the company."""
    task = state.get("task", "")
    if not task:
        return {"research_summary": "No task provided"}
    
    # Search for the company
    search_query = f"{task} company overview recent news"
    search_results = tavily_search.invoke({"query": search_query})
    
    return {
        "raw_search_results": [search_results],  # Internal - stays in scoped state
        "messages": [ToolMessage(content=search_results, tool_call_id="search_1")]
    }

print("✅ Search node defined")


In [ ]:
# Node 2: Summarize node - condenses search results into clean summary
def summarize_node(state: ResearchState) -> dict:
    """Summarize search results into 3 clean paragraphs."""
    if not llm:
        return {"research_summary": "[Mock] Research summary: Company overview, recent developments, and market position."}
    
    raw_results = state.get("raw_search_results", [])
    task = state.get("task", "")
    
    if not raw_results:
        return {"research_summary": f"No search results found for {task}"}
    
    # Combine all search results
    combined_results = "\n\n".join(raw_results)
    
    # Use LLM to create a clean 3-paragraph summary
    prompt = f"""Based on the following search results, write a clean 3-paragraph summary about {task}.

Search Results:
{combined_results}

Write exactly 3 paragraphs:
1. Company Overview (what they do, industry, size)
2. Recent Developments (news, product launches, partnerships)
3. Market Position (competitors, growth, outlook)

Keep it concise and factual. Only use information from the search results."""

    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages)
    summary = response.content
    
    return {
        "research_summary": summary,  # This flows back to SupervisorState!
        "messages": [AIMessage(content=summary)]
    }

print("✅ Summarize node defined")


In [ ]:
# Build the research subgraph
research_graph = StateGraph(ResearchState)

# Add nodes
research_graph.add_node("search", search_node)
research_graph.add_node("summarize", summarize_node)

# Wire edges: START -> search -> summarize -> END
research_graph.add_edge(START, "search")
research_graph.add_edge("search", "summarize")
research_graph.add_edge("summarize", END)

# Compile the subgraph
research_agent = research_graph.compile()

print("✅ Research subgraph compiled!")
print("💡 This subgraph will be used as a single node in the parent graph")


---

# 🔨 Live Build: Step 3 - Writer Agent

The writer agent takes the research summary and creates a structured brief.


In [ ]:
# Writer node - creates structured brief from research summary
def writer_node(state: SupervisorState) -> dict:
    """Write a structured brief from the research summary."""
    if not llm:
        return {"draft": "[Mock] Structured brief:\n\n## Overview\nCompany details...\n\n## Recent Developments\nNews...\n\n## Outreach Signals\nOpportunities..."}
    
    research_summary = state.get("research_summary", "")
    task = state.get("task", "")
    
    if not research_summary:
        return {"draft": f"No research summary available for {task}"}
    
    prompt = f"""Based on the following research summary, write a structured company brief for {task}.

Research Summary:
{research_summary}

Format the brief with these sections:
1. **Overview** - Company description, industry, key facts
2. **Recent Developments** - Latest news, product launches, partnerships
3. **Outreach Signals** - Opportunities for engagement, pain points, growth areas

Keep it professional and actionable."""

    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages)
    draft = response.content
    
    return {
        "draft": draft,
        "messages": [AIMessage(content=f"Brief written for {task}")]
    }

print("✅ Writer node defined")


---

# 🔨 Live Build: Step 4 - Supervisor Agent

The supervisor reads the state and decides who acts next. It never does the work itself.


In [ ]:
# Supervisor node - decides routing
def supervisor_node(state: SupervisorState) -> dict:
    """Supervisor reads state and decides who acts next.
    
    IMPORTANT: Uses boolean state checks to prevent infinite loops.
    Never rely solely on LLM inference for routing decisions.
    """
    # Use boolean checks, not LLM inference (prevents infinite loops)
    research_summary = state.get("research_summary", "").strip()
    draft = state.get("draft", "").strip()
    
    # Debug: Print state to help diagnose issues
    # print(f"DEBUG Supervisor - research_summary: {bool(research_summary)}, draft: {bool(draft)}")
    
    # Make routing decision based on state (deterministic, prevents loops)
    # IMPORTANT: Check draft FIRST if both exist, to prevent re-running researcher
    if draft and len(draft) > 10:  # Draft exists and is substantial
        decision = "FINISH"
    elif research_summary and len(research_summary) > 10:  # Research done, need draft
        decision = "writer"
    else:  # Need research
        decision = "researcher"
    
    # Optional: Use LLM for logging/reasoning (but decision is already made above)
    if llm:
        prompt = f"""You are a supervisor managing a research team.

Current state:
- Task: {state.get('task', 'N/A')}
- Research Summary: {'✅ Complete' if research_summary else '❌ Missing'}
- Draft: {'✅ Complete' if draft else '❌ Missing'}

The routing decision is: {decision}

Confirm this decision is correct."""

        messages = [HumanMessage(content=prompt)]
        try:
            response = llm.invoke(messages)
            reasoning = response.content
        except:
            reasoning = "Decision made based on state checks"
    else:
        reasoning = "Decision made based on state checks (no LLM)"
    
    # Always return lowercase decision (required by routing function)
    decision = decision.lower() if decision != "FINISH" else "FINISH"
    
    return {
        "next": decision,
        "messages": [AIMessage(content=f"Supervisor decision: {decision} ({reasoning[:50]}...)")]
    }

print("✅ Supervisor node defined")
print("💡 Uses deterministic boolean checks to prevent infinite loops")


In [ ]:
# Routing function - conditional edge logic
def route_next(state: SupervisorState) -> Literal["researcher", "writer", "FINISH"]:
    """Route to the next agent based on supervisor's decision.
    
    Handles case-insensitive matching to prevent routing errors.
    """
    next_agent = state.get("next", "researcher")
    
    # Normalize to lowercase for comparison (handle both cases)
    next_agent_lower = str(next_agent).lower().strip()
    
    if next_agent_lower == "finish":
        return "FINISH"
    elif next_agent_lower == "writer":
        return "writer"
    elif next_agent_lower == "researcher":
        return "researcher"
    else:
        # Fallback: default to researcher if decision is unclear
        print(f"⚠️  Warning: Unknown routing decision '{next_agent}', defaulting to researcher")
        return "researcher"

print("✅ Routing function defined")
print("💡 Handles case-insensitive routing decisions")


---

# 🔨 Live Build: Step 5 - Wire the Parent Graph

Now we combine everything: supervisor, research subgraph, and writer into one graph.


In [ ]:
# Build the parent graph
parent_graph = StateGraph(SupervisorState)

# Add nodes
parent_graph.add_node("supervisor", supervisor_node)
parent_graph.add_node("researcher", research_agent)  # Subgraph as a node!
parent_graph.add_node("writer", writer_node)

# Wire edges
parent_graph.add_edge(START, "supervisor")
parent_graph.add_conditional_edges(
    "supervisor",
    route_next,
    {
        "researcher": "researcher",
        "writer": "writer",
        "FINISH": END
    }
)
parent_graph.add_edge("researcher", "supervisor")  # Back to supervisor after research
parent_graph.add_edge("writer", "supervisor")      # Back to supervisor after writing

# Compile the graph
app = parent_graph.compile()

print("✅ Parent graph compiled!")
print("💡 The research subgraph is now a single node in the parent graph")


---

# 🎨 Visualize the Graph

Let's see the structure of our multi-agent system!
image.png

In [ ]:
# Visualize the graph structure
try:
    from IPython.display import Image
    
    # Generate graph visualization
    graph_image = app.get_graph(xray=True).draw_mermaid_png()
    display(Image(graph_image))
    print("✅ Graph visualization generated!")
    print("💡 The 'researcher' node shows as a subgraph with its internal structure")
except Exception as e:
    print(f"⚠️  Could not generate visualization: {e}")
    print("💡 Install graphviz: brew install graphviz (macOS) or apt-get install graphviz (Linux)")
    # Fallback: print graph structure
    print("\nGraph structure:")
    print("START -> supervisor -> [researcher | writer | FINISH]")
    print("  researcher -> supervisor (loop)")
    print("  writer -> supervisor (loop)")


---

# 🚀 Test the System

Let's run the Company Research Assistant!


In [ ]:
# Test the system
def run_research_assistant(company_name: str, max_iterations: int = 10):
    """Run the research assistant for a company.
    
    Args:
        company_name: Name of the company to research
        max_iterations: Maximum number of iterations to prevent infinite loops
    """
    initial_state = {
        "task": company_name,
        "research_summary": "",
        "draft": "",
        "next": "",
        "messages": [HumanMessage(content=f"Research {company_name}")]
    }
    
    print(f"🔍 Researching: {company_name}\n")
    print("=" * 60)
    
    # Stream execution to see each step and accumulate state
    accumulated_state = initial_state.copy()
    iteration_count = 0
    
    for step in app.stream(initial_state):
        iteration_count += 1
        
        # Safety check: prevent infinite loops
        if iteration_count > max_iterations:
            print(f"\n⚠️  WARNING: Stopped after {max_iterations} iterations to prevent infinite loop")
            print("   This usually means the supervisor is not detecting completion correctly.")
            print(f"   Current state - research_summary: {bool(accumulated_state.get('research_summary'))}, draft: {bool(accumulated_state.get('draft'))}")
            break
        
        for node, output in step.items():
            print(f"\n📍 Node: {node} (iteration {iteration_count})")
            if node == "supervisor":
                next_decision = output.get("next", "")
                print(f"   Decision: {next_decision}")
                # Debug: show what supervisor sees
                current_research = bool(accumulated_state.get("research_summary", "").strip())
                current_draft = bool(accumulated_state.get("draft", "").strip())
                print(f"   State check - research: {current_research}, draft: {current_draft}")
            elif node == "researcher":
                summary = output.get("research_summary", "")
                if summary:
                    print(f"   Research Summary: {summary[:200]}...")
            elif node == "writer":
                draft = output.get("draft", "")
                if draft:
                    print(f"   Draft: {draft[:200]}...")
            
            # Accumulate state updates (merge output into accumulated_state)
            for key, value in output.items():
                if key == "messages" and isinstance(value, list):
                    # Append messages
                    if "messages" not in accumulated_state:
                        accumulated_state["messages"] = []
                    accumulated_state["messages"].extend(value)
                else:
                    # Overwrite other fields (including empty strings - they're valid updates)
                    # This ensures draft, research_summary, etc. persist
                    accumulated_state[key] = value
            
            # Check if we've reached FINISH
            if output.get("next") == "FINISH":
                print("\n✅ Supervisor decided to FINISH")
                break
    
    # Use accumulated state as final state
    final_state = accumulated_state
    
    print("\n" + "=" * 60)
    print("\n✅ Final Output:\n")
    
    if final_state:
        draft = final_state.get("draft", "")
        if draft:
            print(draft)
        else:
            print("No draft generated")
            print(f"Debug - Final state keys: {list(final_state.keys())}")
            print(f"Debug - Draft value: {repr(final_state.get('draft', 'NOT FOUND'))}")
            print(f"Debug - Research summary: {repr(final_state.get('research_summary', 'NOT FOUND'))}")
    
    return final_state

# Test with a company
print("Ready to test! Run: run_research_assistant('OpenAI')")


In [ ]:
# Example: Research OpenAI
result = run_research_assistant("OpenAI")


---

# 🐛 What Breaks & How To Fix It

## Common Issues and Solutions

| Issue | Cause | Fix |
|-------|-------|-----|
| **Infinite loops** | Vague supervisor prompt, never reaches FINISH | Use boolean state checks, not LLM inference |
| **State schema mismatch** | Parent calls it `task`, subgraph expects `query` | Print state keys at every node during development |
| **Shared state noise** | All agents appending raw tool results to messages | Only write clean outputs to shared state, keep internals scoped |
| **Subgraph output not reaching parent** | Field names don't match between schemas | Field names and types must be identical in both state schemas |

**Always debug with `app.stream()` not `app.invoke()`**


? w

In [ ]:
# Debug helper: Print state at each step
def debug_stream(company_name: str):
    """Stream execution with detailed state inspection."""
    initial_state = {
        "task": company_name,
        "research_summary": "",
        "draft": "",
        "next": "",
        "messages": [HumanMessage(content=f"Research {company_name}")]
    }
    
    print(f"🔍 Debugging: {company_name}\n")
    print("=" * 60)
    
    step_count = 0
    for step in app.stream(initial_state):
        step_count += 1
        for node, output in step.items():
            print(f"\n📍 Step {step_count} - Node: {node}")
            print(f"   State keys: {list(output.keys())}")
            
            # Check for state mismatches
            if "task" in output:
                print(f"   ✅ task: {output['task']}")
            if "research_summary" in output:
                summary = output["research_summary"]
                print(f"   ✅ research_summary: {'Present' if summary else 'Missing'}")
            if "draft" in output:
                draft = output["draft"]
                print(f"   ✅ draft: {'Present' if draft else 'Missing'}")
            if "next" in output:
                print(f"   ✅ next: {output['next']}")
    
    print("\n" + "=" * 60)
    print(f"✅ Completed in {step_count} steps")

print("✅ Debug helper defined")
print("💡 Use debug_stream('CompanyName') to see detailed state flow")


---

# 📚 Homework

## Extend the Company Research Assistant with a Third Agent

Choose one of the following options:

### Option A — Critic Agent

Reviews the draft, scores it 1-10. If below 7, routes back to writer for revision.

**Implementation hints:**
- Add `score: int` to `SupervisorState`
- Create `critic_node` that scores the draft
- Update routing logic: `supervisor -> critic -> [writer (if score < 7) | FINISH]`

### Option B — Outreach Agent

Takes the completed brief, drafts a personalized cold email using research signals.

**Implementation hints:**
- Add `outreach_email: str` to `SupervisorState`
- Create `outreach_node` that generates email
- Update routing: `writer -> outreach -> FINISH`

### Option C — Comparison Agent

Accept two company names. Run two research subgraphs in parallel. Compare side-by-side.

**Implementation hints:**
- Modify `SupervisorState` to accept `task2: str`
- Create two research subgraphs: `research_agent_1` and `research_agent_2`
- Use parallel execution or sequential with comparison node

### Deliverables

1. ✅ Working code on GitHub
2. ✅ Screenshot of output
3. ✅ Brief explanation of your implementation

---

## 🎁 Bonus: Post on LinkedIn

```
Just built a multi-agent AI system in LangGraph.

A supervisor that coordinates specialist agents to 
research companies and write briefs — automatically.

Week 3 of [course name]. We went from a single ReAct 
agent to a full orchestrated team.

Graph 👇 [screenshot]

The thing that clicked for me: [your insight]

#LangGraph #MultiAgent #AIEngineering #BuildInPublic
```

Tag the cohort. Best post gets featured in Week 4 recap.

---

## 🔗 Resources

- [LangGraph Official Documentation](https://docs.langchain.com/oss/python/langgraph/overview)
- [LangGraph Subgraphs](https://langchain-ai.github.io/langgraph/how-tos/subgraphs/)
- [LangGraph State Management](https://langchain-ai.github.io/langgraph/concepts/low_level/#state)
- [LangSmith](https://smith.langchain.com) — Debug & trace your graphs

---

**Happy Building! 🎉**
